In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time, datetime 

In [2]:

pd.set_option('display.max_columns', None)

# 定义数据库连接

In [3]:
def query(sql,
          host="rm-wz930e5269fur1ht1mo.mysql.rds.aliyuncs.com",
          user="wxz",
          password="5JRcY9SaiepVlIq7iuPo",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入订单数据

In [4]:
sql1 = '''
SELECT 
tt.order_number '订单号', tt.true_name '姓名', tt.user_mobile '手机号', tt.product_name '产品名称', tt.create_time '下单日期', tt.refund_date '预计还款日期', tt.reality_refund_date '实际还款日期', 
tt.money '应付金额', tt.real_pay_money '实付金额', tt.min_date, tt.max_date, tt.diff_date
FROM (
SELECT  t.order_number,t.true_name,t.user_mobile,t.product_name,t.create_time, t.refund_date,t.reality_refund_date,t.money,t.real_pay_money,t.min_date,
case when max_date>=DATE_ADD(CURRENT_DATE, INTERVAL -1 DAY) then CURRENT_DATE else t.max_date END max_date,
case when t.min_date=t.max_date then DATEDIFF(CURRENT_DATE, t.min_date) 
			when max_date>CURRENT_DATE then DATEDIFF(CURRENT_DATE, t.min_date) 
			else DATEDIFF(t.max_date, t.min_date) END diff_date
FROM (
SELECT -- 查询总体订单和去重以及剔除商家
om.order_number,tmu.true_name,om.user_mobile,tod.product_name,om.create_time,tos.refund_date,tos.reality_refund_date,tos.money,tos.real_pay_money
,min(tos.refund_date)over(partition by om.order_number) min_date
,max(tos.refund_date)over(partition by om.order_number) max_date
FROM db_digua_business.t_postlease_receivables_monitoring   tprm 
LEFT JOIN db_digua_business.t_order om on tprm.order_id = om.id 
LEFT JOIN db_digua_business.t_merchant tmer on tmer.id = tprm.merchant_id
LEFT JOIN db_digua_business.t_order_stages tos on om.id=tos.order_id
-- LEFT JOIN db_digua_business.t_order_relet tor on om.id=tor.order_id
LEFT JOIN db_digua_business.t_order_details tod on tod.order_id=om.id 
LEFT JOIN db_digua_business.t_member_user tmu on tmu.mobile=om.user_mobile
-- LEFT JOIN db_digua_business.t_order_relet tor on tor.order_id=om.id
WHERE om.`status` in (2, 3, 4) 
AND tmer.shop_type !=2  -- 剔除企业 1、自营店铺；2、入驻店铺；3、代运营店铺'
AND tprm.merchant_id  not in (15,99)  -- 剔除CPS渠道合作 汇客好租
AND tprm.model_number not like '%探路者%'
-- AND tos.refund_date = DATE_ADD(CURRENT_DATE,INTERVAL +1 DAY)
AND tos.real_pay_money<tos.money
and tprm.activity_id != 335
and om.has_actual!=1
) t
) tt
WHERE diff_date<=0 and refund_date>=CURRENT_DATE and refund_date<=DATE_ADD(CURRENT_DATE, INTERVAL +1 DAY)

'''
df_order = query(sql1)
df_order.shape

(217, 12)

In [5]:
sql2 = '''
SELECT tt.order_number '订单号', tt.true_name '姓名', tt.user_mobile '手机号', tt.product_name '产品名称', tt.create_time '下单日期', tt.relet_start_time '预计还款日期', tt.money '应付金额', tt.min_time FROM (
SELECT t.order_number, t.true_name, t.user_mobile, t.product_name, t.create_time, t.relet_start_time, t.money,
min(relet_start_time)over(partition by t.order_id) min_time FROM(
SELECT om.order_number, tor.order_id, tmu.true_name, om.user_mobile, tod.product_name, om.create_time, tor.relet_start_time, tor.money FROM db_digua_business.t_postlease_receivables_monitoring   tprm
LEFT JOIN db_digua_business.t_order om on tprm.order_id = om.id 
LEFT JOIN db_digua_business.t_merchant tmer on tmer.id = tprm.merchant_id
LEFT JOIN db_digua_business.t_order_relet tor on tprm.order_id=tor.order_id
LEFT JOIN db_digua_business.t_order_details tod on tod.order_id=om.id 
LEFT JOIN db_digua_business.t_member_user tmu on tmu.mobile=om.user_mobile
WHERE tmer.shop_type !=2  -- 剔除企业 1、自营店铺；2、入驻店铺；3、代运营店铺'
AND tprm.merchant_id  not in (15,99)  -- 剔除CPS渠道合作 汇客好租
AND tprm.model_number not like '%探路者%'
and cycle_status = 1
and tprm.activity_id != 335
and om.has_actual!=1
) t
) tt
WHERE min_time in (CURRENT_DATE, DATE_ADD(CURRENT_DATE, INTERVAL +1 DAY)) and relet_start_time in (CURRENT_DATE, DATE_ADD(CURRENT_DATE, INTERVAL +1 DAY))
'''
df_order2 = query(sql2)
df_order2.shape

(47, 8)

In [6]:
sql3 = '''
SELECT yoo.order_number '订单号', tmu.true_name '姓名', om.user_mobile '手机号', tod.product_name '产品名称', om.create_time '下单日期', yoo.overdue_time '预计还款日期', yoo.overdue_money '应付金额'
FROM db_digua_business.t_order om
LEFT JOIN db_rent.ya_order_overdue yoo on yoo.order_id=om.id
LEFT JOIN db_digua_business.t_order_details tod on tod.order_id=om.id 
LEFT JOIN db_digua_business.t_member_user tmu on tmu.mobile=om.user_mobile
WHERE DATE_FORMAT(om.create_time,'%Y-%m')=DATE_FORMAT(DATE_ADD(CURRENT_DATE,INTERVAL -1 MONTH), '%Y-%m')  and yoo.type = 0
'''
df_order3 = query(sql3)
df_order3.shape

(36, 7)

In [7]:
sql4 = '''
SELECT  t.channel_name ,t.activity_name, t.total_describes, t.rejected, t.result, t.order_id,
t.order_number '订单号', t.true_name '姓名', t.user_mobile '手机号', t.product_name '产品名称', t.create_time '下单日期', t.refund_date '预计还款日期', t.reality_refund_date '实际还款日期', 
t.money '应付金额', t.real_pay_money '实付金额', diff_date '逾期天数',t.sort,t.order_method, t.status2
FROM (
SELECT -- 查询总体订单和去重以及剔除商家
cc.`name` as channel_name
,pa.`name` as activity_name, tprm.order_id
,om.order_number,tmu.true_name,om.user_mobile,tod.product_name,om.create_time,tos.refund_date,tos.reality_refund_date,tos.money,tos.real_pay_money, tos.sort,
case 	when tos.reality_refund_date is null then DATEDIFF(CURRENT_DATE, tos.refund_date)
			when DATE_FORMAT(tos.reality_refund_date,'%Y-%m-%d')>DATE_FORMAT(tos.refund_date,'%Y-%m-%d') then DATEDIFF(tos.reality_refund_date,tos.refund_date)
			else null END diff_date
,om.order_method,top.total_describes
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,replace(case when JSON_VALID(tori.decision_result) THEN JSON_EXTRACT(tori.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tori.decision_result) THEN JSON_EXTRACT(tori.decision_result, '$.result') end,'"','') as result 
FROM  db_digua_business.t_order om
LEFT JOIN db_digua_business.t_postlease_receivables_monitoring   tprm on tprm.order_id = om.id 
LEFT JOIN db_digua_business.t_merchant tmer on tmer.id = tprm.merchant_id
LEFT JOIN db_digua_business.t_order_stages tos on om.id=tos.order_id
-- LEFT JOIN db_digua_business.t_order_relet tor on om.id=tor.order_id
LEFT JOIN db_digua_business.t_order_details tod on tod.order_id=om.id 
LEFT JOIN db_digua_business.t_member_user tmu on tmu.mobile=om.user_mobile
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top on om.id = top.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
left join db_digua_business.t_order_risk tori on om.id = tori.order_id
WHERE om.`status` in (2, 3, 4) 
AND tmer.shop_type !=2  -- 剔除企业 1、自营店铺；2、入驻店铺；3、代运营店铺'
AND tprm.merchant_id  not in (15,99)  -- 剔除CPS渠道合作 汇客好租
AND tprm.model_number not like '%探路者%'
AND DATE_FORMAT(om.create_time,'%Y-%m')>=DATE_FORMAT(DATE_ADD(CURRENT_DATE, INTERVAL -3 MONTH), '%Y-%m')
AND DATE_FORMAT(om.create_time,'%Y-%m')<=DATE_FORMAT(DATE_ADD(CURRENT_DATE, INTERVAL -1 MONTH), '%Y-%m')
AND tos.refund_date <= CURRENT_DATE
and tprm.activity_id != 335
) t
WHERE DATE_FORMAT(t.refund_date,'%Y-%m')=DATE_FORMAT(CURRENT_DATE,'%Y-%m')
'''
df_order4 = query(sql4)
df_order4.shape

(2911, 19)

In [8]:
dd_date = '2023-10'
sql5 = f'''
SELECT 
tprm.order_number '订单号', tprm.search_time '下单月份', tprm.model_number '型号', tprm.specification '规格', tprm.order_create_time '下单时间', om.create_time '创建时间', tprm.rembursement_status '还款状态', 
tprm.paid_periods '已还期数', tprm.end_date '租期结束日期', tprm.order_finish_date '订单完成时间'
FROM db_digua_business.t_postlease_receivables_monitoring tprm
LEFT JOIN db_digua_business.t_order om ON tprm.order_id=om.id
WHERE tprm.search_time='{dd_date}'
and tprm.platform_package != '租完即送'
and om.merchant_id not in (15,18,21,30,31,33,79,85,99,107,124)
'''
df_order5 = query(sql5)
df_order5.shape

(1030, 10)

# 合并数据

In [9]:
df = df_order.copy()
df2 = df_order2.copy()
df3 = df_order3.copy()
df4 = df_order4.copy()
df5 = df_order5.copy()
# df.order_id.notnull().sum()

In [10]:

df4["取消原因"]=df4["total_describes"].str.split("客户申请取消：").str[1].astype(str).str.split("$").str[0].str.strip()
df4["电审拒绝原因"]=df4["total_describes"].str.split("审核不通过：").str[1].astype(str).str.split("$").str[0].str.strip()
df4["无法联系原因"]=df4["total_describes"].str.split("用户无法联系：").str[1].astype(str).astype(str).str.split("$").str[0].str.strip()


df4.loc[:,"进件"]=np.where((df4["status2"]=="待支付")|(df4["status2"]=="订单取消"),"未进件","进件")
df4["进件"].value_counts()

# %%
df4.loc[:,"来源渠道"]=df4["channel_name"].fillna("未知渠道")

# 渠道归属

In [11]:
df4['拒绝理由'] = df4["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')
df4['拒绝理由'].value_counts()

拒绝理由
                             2814
命中云商分低于515强拒(芝麻综合风险等级0豁免)      97
Name: count, dtype: int64

In [12]:
def qudao_type(a,b,c):
    a = str(a)
    b = str(b)
    if "租物" in b:
        return "芝麻租物"
    elif "芝麻" in b:
        return "芝麻租物"
    elif "抖音" in b:
        return "抖音渠道"
    elif "搜索" in a:
        return "搜索渠道"
    elif "租物" in a:
        return "芝麻租物"
    elif "芝麻" in a:
        return "芝麻租物"
    elif c == 1:
        return "芝麻租物"   
    elif "叮咚直播" in a:
        return "叮咚直播"
    elif "租瓜直播2号" in a:
        return "租瓜直播2号"
    elif "租瓜直播" in a:
        return "租瓜直播"
    elif "直播" in a:
        return "小舞直播"
    elif "繁星" in a:
        return "繁星"
    elif "生活号" in a:
        return "生活号"
    elif "群" in a:
        return "支付宝社群"
    elif "付费灯火" in a:
        return "付费灯火"
    else :
        return a
    

# %%
df4.loc[:,"归属渠道"]=df4.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)
df4["归属渠道"].value_counts()

归属渠道
芝麻租物                         2026
搜索渠道                          589
抖音渠道                           90
八派信息                           55
我的小程序入口                        41
付费灯火                           27
单人聊天会话中的小程序消息卡片（分享）            11
付费邦道-苹果旗舰                       8
生活号                             8
支付宝社群                           7
九州信息                            6
付费流量(通过商家数字推广平台，灯火等投放的广告)       6
ocpc首页推荐交易                      5
小舞直播                            5
小程序快窗                           5
小程序商家消息（服务提醒）                   4
其他渠道场景渠道。                       4
付费邦道-首页                         3
付费邦道-降价了                        3
美仑美奂                            2
未知渠道                            2
白名单客户                           1
付费邦道-华为专区                       1
邦道-销售线索                         1
支付宝 push 消息（同1014）              1
Name: count, dtype: int64

In [13]:
# df4.columns

In [14]:
# #删除重复单号
# df4.drop_duplicates(subset=["order_id"],inplace=True)
# #去刷单订单
# df4.drop(df4[df4['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
# df4.drop(df4[df4['activity_name']=="1000单秘密计划"].index,inplace=True)
# df4.drop(df4[df4['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
# df4.drop(df4[df4['activity_name']=="1000单曙光计划"].index,inplace=True)
# df4.drop(df4[df4['activity_name']=="线下门店3个月试行"].index,inplace=True)

# df4.dropna(subset=["status2"],axis=0,inplace=True)


# #删除重复订单
# df4.drop_duplicates(subset=["order_id"],inplace=True)
# df4.drop_duplicates(subset=["姓名","手机号","id_card_num","下单日期"],keep="last",inplace=True)


# df4.drop(df4[df4['姓名'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
# ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文"])].index,inplace=True)

In [15]:
# 判断 前置拦截   机审强拒   出库前风控强拒
df4['是否前置拦截'] = np.where((df4.result.str.contains('id_card不得为空') )|(df4.result.str.contains('mobile校验不通过')) |(df4.result.str.contains('name校验不通过'))
                        |(df4.result.str.contains('年龄超过49岁或低于18岁'))|(df4.result=='风控未通过')|(df4.result=='黑名单用户'),1,0)
df4['是否机审强拒'] = np.where((df4.result.str.contains('风控拒绝') & (~df4.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
df4['是否出库前风控强拒'] = np.where((df4.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df4.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df4.result.str.contains('黑名单用户').fillna(False))) ,1,0)


In [16]:
def reject_type(a,b,c,d,e,f,g,h,i,j):
    if h==1 and b=="未进件":
        return "前置拦截"
    elif i==1 and b=="进件":
        return "机审强拒"    
    #抖音渠道
    # elif h==1 and b=="进件":
    #     return "机审强拒"  
    elif  b=="进件" and c is not np.nan:
        return "人审拒绝"
    elif  b=="进件" and d is not np.nan:
        return "客户取消"
    elif  b=="进件" and f is not np.nan:
        return "无法联系"
    elif  b=="进件" and j==1 :
        return "出库前风控强拒"
    elif  b=="进件" and "待审核" in e:
        return "待审核"
    elif  b=="进件" and "待发货" in e:
        return "出库"
    elif  b=="进件" and "待收货" in e:
        return "出库"
    elif b=="进件" and "租赁中" in e:
        return "出库"
    elif  b=="进件" and "已完成" in e:
        return "出库"
    elif a is np.nan and b=="未进件":
        return "未进件"
    elif  b=="未进件":
        return "进件前取消"
    else :
        return e

df4.loc[:,"审核状态"]=df4.apply(lambda x: reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)#,x["无法联系原因"]
df4["审核状态"].value_counts()  

审核状态
人审拒绝    2911
Name: count, dtype: int64

In [17]:

df4_j = df4[df4["进件"]=="进件"]
df4_j.shape
df4["是否进件"] = np.where(df4["进件"]=='进件',1,0)

In [18]:
df_all = pd.concat([df, df2])
df_all
indx = round(len(df_all)/10)
df_cs = df_all[:indx]
df_xs = df_all[indx:]
df_all

,订单号,姓名,手机号,产品名称,下单日期,预计还款日期,实际还款日期,应付金额,实付金额,min_date,max_date,diff_date,min_time
0,A2023111103401157,陈子健,19141002689,iPhone15ProMax 全新国行 顺丰包邮,2023-11-11 03:40:11,2024-10-14,None,1085.68,0.0,2024-10-14,2024-10-14,0.0,NaT
1,A2023111104495436,汪锐,13224084920,iPhone15Pro 全新国行 顺丰包邮,2023-11-11 04:49:55,2024-10-14,None,524.38,0.0,2024-10-14,2024-10-14,0.0,NaT
2,A2023111109465499,陈倩倩,18692606001,iPhone15 全新国行 24小时内发货 6.1英寸屏幕 顺丰包邮,2023-11-11 09:46:54,2024-10-14,None,440.74,0.0,2024-10-14,2024-10-14,0.0,NaT
3,A20231112005735116,杨雪莹,18578226702,iPhone15Pro 全新国行 顺丰包邮,2023-11-12 00:57:35,2024-10-15,None,734.55,0.0,2024-10-15,2024-10-14,-1.0,NaT
4,A20231112135355143,欧阳紫萱,18579030926,iPhone15ProMax 全新国行 顺丰包邮,2023-11-12 13:53:56,2024-10-15,None,658.01,0.0,2024-10-15,2024-10-14,-1.0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,A2023101303201333,李观权,14775436914,iPhone15Pro 全新国行 顺丰包邮,2023-10-13 03:20:14,2024-10-15,NaN,598.57,NaN,NaT,NaT,NaN,2024-10-15
43,A2023101308375884,王会兰,15515451860,iPhone14ProMax 国行正品 顺丰包邮 现货速发 高通过率,2023-10-13 08:37:58,2024-10-15,NaN,837.80,NaN,NaT,NaT,NaN,2024-10-15
44,A2023101309145935,袁兴丽,18287126311,iPhone 14 Pro Max 全新国行 全国联保 顺丰包邮 现货速发,2023-10-13 09:14:59,2024-10-15,NaN,583.54,NaN,NaT,NaT,NaN,2024-10-15
45,A2023101312534685,李豪博,17638297427,iPhone14ProMax 全新国行 全国联保 顺丰包邮 现货速发,2023-10-13 12:53:46,2024-10-15,NaN,641.93,NaN,NaT,NaT,NaN,2024-10-15


In [19]:
now_date = datetime.now().strftime('%Y-%m-%d')
next_date = (pd.to_datetime(now_date) + pd.Timedelta(days=1)).strftime('%Y-%m-%d')
# df_xs1 = df_xs[df_xs['预计还款日期']==now_date][['订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '应付金额']]
# df_xs2 = df_xs[df_xs['预计还款日期']==next_date][['订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '应付金额']]
# df_xs2
df_xs = df_xs[['订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '应付金额']].sort_values(by='预计还款日期')
df_xs

,订单号,姓名,手机号,产品名称,下单日期,预计还款日期,应付金额
26,A2024011112385371,张丹,13588234525,iPhone15ProMax 全新国行 顺丰包邮,2024-01-11 12:38:54,2024-10-14,666.29
149,A2024081111140537,简小为,14775801775,iPhone15Pro 租金全网可比 全新 无锁 非监管机 顺丰包邮,2024-08-11 11:14:06,2024-10-14,457.98
148,A2024081109461887,陈焱,15955165067,iPhone15Pro 租金全网可比 全新 无锁 非监管机 顺丰包邮,2024-08-11 09:46:19,2024-10-14,356.17
147,A2024081109420377,王爽爽,17551054549,iPhone15ProMax 降价了 全新国行 顺丰包邮 不查征信,2024-08-11 09:42:03,2024-10-14,526.01
146,A202408110303334,刘猛,15666624752,iPhone15Pro 租金全网可比 全新 无锁 非监管机 顺丰包邮,2024-08-11 03:03:33,2024-10-14,356.17
...,...,...,...,...,...,...,...
154,A2024081202530170,崔晏瑞,13038888180,首月1元 iPhone15ProMax 全新国行 顺丰包邮,2024-08-12 02:53:02,2024-10-15,454.42
155,A2024081203294639,徐栋飞,13250521381,iPhone15Pro 租金全网可比 全新 无锁 非监管机 顺丰包邮,2024-08-12 03:29:46,2024-10-15,348.23
156,A202408120501331,张浩川,13152222957,iPhone15ProMax 降价了 全新国行 顺丰包邮 低租金高通过,2024-08-12 05:01:34,2024-10-15,627.85
142,A2024071222045511,李秋阳,15196792096,iPhone15 非监管机 低租金低买断 全新国行 顺丰包邮,2024-07-12 22:04:55,2024-10-15,394.29


In [20]:
# df_cs1 = df_cs[df_cs['预计还款日期']==now_date][['订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '应付金额']]
# df_cs2 = df_cs[df_cs['预计还款日期']==next_date][['订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '应付金额']]
# df_cs1
df_cs = df_cs[['订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '应付金额']].sort_values(by='预计还款日期')
df_cs

,订单号,姓名,手机号,产品名称,下单日期,预计还款日期,应付金额
0,A2023111103401157,陈子健,19141002689,iPhone15ProMax 全新国行 顺丰包邮,2023-11-11 03:40:11,2024-10-14,1085.68
24,A2024011111213945,周高建,18685482326,iPhone15ProMax 全新国行 顺丰包邮,2024-01-11 11:21:39,2024-10-14,582.98
11,A20231211174721120,梅小飞,15814083825,iPhone15Pro 全新国行 顺丰包邮,2023-12-11 17:47:22,2024-10-14,599.65
10,A20231211165031187,王玉龙,13151234222,iPhone15Pro 全新国行 顺丰包邮,2023-12-11 16:50:31,2024-10-14,599.65
9,A2023121116113147,罗玉琴,17608558559,iPhone15Pro 全新国行 顺丰包邮,2023-12-11 16:11:32,2024-10-14,599.65
8,A202312111401203,李文成,17643302625,iPhone15ProMax 全新国行 顺丰包邮,2023-12-11 14:01:21,2024-10-14,1082.82
7,A2023121113495282,石红燕,15923458758,iPhone15ProMax 全新国行 顺丰包邮,2023-12-11 13:49:52,2024-10-14,666.29
25,A2024011112131728,谢学城,15255018432,iPhone15ProMax 全新国行 顺丰包邮,2024-01-11 12:13:18,2024-10-14,582.98
5,A2023121100325722,刘郢诚,17612080015,iPhone15Pro 全新国行 顺丰包邮,2023-12-11 00:32:58,2024-10-14,598.55
2,A2023111109465499,陈倩倩,18692606001,iPhone15 全新国行 24小时内发货 6.1英寸屏幕 顺丰包邮,2023-11-11 09:46:54,2024-10-14,440.74


In [21]:

# with pd.ExcelWriter(f'F:/需求/到期订单/到期订单_信审_{now_date}-{next_date}.xlsx', engine='openpyxl') as writer:
#     df_xs.to_excel(writer, sheet_name=now_date, index=False)

In [22]:
# with pd.ExcelWriter(f'F:/需求/到期订单/到期订单_催收_{now_date}-{next_date}.xlsx', engine='openpyxl') as writer:
#     df_cs.to_excel(writer, sheet_name=now_date, index=False)

# 逾期订单天数

In [23]:
# df3['逾期天数'] = datetime.now() - df3['预计还款日期']
# df3
df3['逾期天数'] = pd.to_datetime(datetime.now().strftime('%Y-%m-%d'))- df3['预计还款日期']
# df3.groupby('逾期天数').agg({'订单号': 'count'})
# df3

# 下单时间上月截止至今日的订单

## 到期订单

In [24]:
df4.columns

Index(['channel_name', 'activity_name', 'total_describes', 'rejected',
       'result', 'order_id', '订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期',
       '实际还款日期', '应付金额', '实付金额', '逾期天数', 'sort', 'order_method', 'status2',
       '取消原因', '电审拒绝原因', '无法联系原因', '进件', '来源渠道', '拒绝理由', '归属渠道', '是否前置拦截',
       '是否机审强拒', '是否出库前风控强拒', '审核状态', '是否进件'],
      dtype='object')

In [25]:
df4['下单月份'] = df4['下单日期'].dt.strftime('%Y-%m')
df4['预计还款月份'] = df4['预计还款日期'].dt.strftime('%Y-%m')
create_month = df4['下单月份'].to_list()
# create_month

In [56]:
Today1 = datetime.now().strftime('%Y-%m-%d')
def order_end(df, month1, month2):
    date_diff = (pd.to_datetime(Today1).to_period('M')-pd.to_datetime(month2).to_period('M')).n
    td = Today1
    if date_diff==0:
        td
    elif date_diff==1:
        td = (pd.to_datetime(Today1)-timedelta(days=int(datetime.now().strftime('%d')))).strftime('%Y-%m-%d')
    else:
        td = (pd.to_datetime(Today1)-timedelta(days=int(datetime.now().strftime('%d')))-pd.DateOffset(months=date_diff-1)).strftime('%Y-%m-%d')
    df4_dq = df[(df['下单月份']==month1)&(df['预计还款月份']==month2)]
    # 月总订单
    dd_dq_month = df4_dq['订单号'].count()
    # 逾期1+订单
    df_yq_1= df4_dq[(df4_dq['逾期天数']>=1)]
    dd_yq_1 = df_yq_1['订单号'].count()
    # 逾期4+订单
    df_yq_4= df4_dq[(df4_dq['逾期天数']>=4)]
    dd_yq_4 = df_yq_4['订单号'].count()
    # 逾期7+订单
    df_yq_7= df4_dq[(df4_dq['逾期天数']>=7)]
    dd_yq_7 = df_yq_7['订单号'].count()
    # 逾期15+订单
    df_yq_15= df4_dq[(df4_dq['逾期天数']>=15)]
    dd_yq_15 = df_yq_15['订单号'].count()

    # 今日到期订单
    df_dq_today = df4_dq[df4_dq['预计还款日期']==td]
    dd_dq_today_count = df_dq_today['订单号'].count()
    # 今日到期已还款
    dq_yhk = df_dq_today[df_dq_today['实付金额'] >= df_dq_today['应付金额']]
    dd_dq_yhk = dq_yhk['订单号'].count()
    # 今日到期未还款
    dq_whk = df_dq_today[df_dq_today['实付金额'] < df_dq_today['应付金额']]
    dd_dq_whk = dq_whk['订单号'].count()
    dd_dq_whk

    # 汇总
    dd_all = pd.DataFrame({
    '截止日期': [td],
    '本月到期订单': [dd_dq_month],
    '今日到期订单': [dd_dq_today_count],
    '逾期1+订单': [dd_yq_1],
    '逾期4+订单': [dd_yq_4],
    '逾期7+订单': [dd_yq_7],
    '逾期15+订单': [dd_yq_15],
    '今日到期已还款订单': [dd_dq_yhk],
    '今日到期未还款订单': [dd_dq_whk]
    })
    dd_all['本月1+逾期率'] = dd_yq_1/dd_dq_month * 100
    dd_all['本月1+逾期率'] = dd_all['本月1+逾期率'].apply(lambda x: f'{x: .2f}%')
    dd_all['本月4+逾期率'] = dd_yq_4/dd_dq_month * 100
    dd_all['本月4+逾期率'] = dd_all['本月4+逾期率'].apply(lambda x: f'{x: .2f}%')
    dd_all['本月7+逾期率'] = dd_yq_7/dd_dq_month * 100
    dd_all['本月7+逾期率'] = dd_all['本月7+逾期率'].apply(lambda x: f'{x: .2f}%')
    dd_all['本月15+逾期率'] = dd_yq_15/dd_dq_month * 100
    dd_all['本月15+逾期率'] = dd_all['本月15+逾期率'].apply(lambda x: f'{x: .2f}%')
    dd_all['今日到期已还款率'] = dd_dq_yhk/dd_dq_today_count * 100
    dd_all['今日到期已还款率'] = dd_all['今日到期已还款率'].apply(lambda x: f'{x:.2f}%')
    dd_all['今日到期未还款率'] = dd_dq_whk/dd_dq_today_count * 100
    dd_all['今日到期未还款率'] = dd_all['今日到期未还款率'].apply(lambda x: f'{x:.2f}%')
    dd_all = dd_all[['截止日期','本月到期订单','逾期1+订单','逾期4+订单','逾期7+订单','逾期15+订单','本月1+逾期率','本月4+逾期率','本月7+逾期率','本月15+逾期率','今日到期订单','今日到期已还款订单','今日到期已还款率','今日到期未还款订单','今日到期未还款率']]
    dd_all



    return df4_dq, dd_all


In [61]:
def qd_info(df, qd, qd_name):    
    df = df[df[qd]==qd_name]
    df_count = df['订单号'].count()
    df_jj = df[df['是否进件']==1]
    df_jj = df_jj[['订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '实际还款日期', '应付金额', '实付金额']]
    df_jj_count = df_jj['订单号'].count()

    df_yq = df[df['逾期天数']>=1]
    df_yq = df_yq[['订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '实际还款日期', '应付金额', '实付金额']]
    df_yq_count = df_yq['订单号'].count()
    df_yq
    return df_yq

In [60]:
df4_dq, dd_all = order_end(df4, '2024-09', '2024-10')
dd_all

,截止日期,本月到期订单,逾期1+订单,逾期4+订单,逾期7+订单,逾期15+订单,本月1+逾期率,本月4+逾期率,本月7+逾期率,本月15+逾期率,今日到期订单,今日到期已还款订单,今日到期已还款率,今日到期未还款订单,今日到期未还款率
0,2024-10-14,1046,63,24,12,0,6.02%,2.29%,1.15%,0.00%,94,84,89.36%,10,10.64%


In [62]:
qd_info(df4_dq, '归属渠道', '八派信息')

,订单号,姓名,手机号,产品名称,下单日期,预计还款日期,实际还款日期,应付金额,实付金额
1961,A20240902122005103,王睿,15790806674,iPhone15ProMax 全新国行 顺丰包邮,2024-09-02 12:20:06,2024-10-05,NaT,1082.84,0.0
2264,A2024090611182986,张瑞杰,13916000798,iPhone15 全新国行 6.1英寸屏幕,2024-09-06 11:18:29,2024-10-09,NaT,757.97,0.0


In [63]:
qd_info(df4_dq, '归属渠道', '九州信息')

,订单号,姓名,手机号,产品名称,下单日期,预计还款日期,实际还款日期,应付金额,实付金额
2010,A2024090220291563,苏义付,18095377718,iPhone15ProMax 全新国行 顺丰包邮,2024-09-02 20:29:15,2024-10-05,2024-10-06 21:26:34,1082.84,1082.84


In [29]:
dd_dq_month = df4['订单号'].count()
dd_dq_month

np.int64(2911)

## 逾期1+订单

In [30]:
df_yq_1= df4[(df4['逾期天数']>=1)]
dd_yq_1 = df_yq_1['订单号'].count()
dd_yq_1

np.int64(342)

## 逾期4+订单

In [31]:
df_yq_4= df4[(df4['逾期天数']>=4)]
dd_yq_4 = df_yq_4['订单号'].count()
dd_yq_4

np.int64(178)

## 逾期7+订单

In [32]:
df_yq_7= df4[(df4['逾期天数']>=7)]
dd_yq_7 = df_yq_7['订单号'].count()
dd_yq_7

np.int64(108)

## 逾期15+订单

In [33]:
df_yq_15= df4[(df4['逾期天数']>=15)]
dd_yq_15 = df_yq_15['订单号'].count()
dd_yq_15

np.int64(0)

# 今日到期订单

In [34]:
Today1 = datetime.now().strftime('%Y-%m-%d')


In [35]:
df_dq= df4[df4['预计还款日期']==Today1]
dd_dq = df_dq['订单号'].count()
# df_dq

## 到期已还款

In [36]:
dq_yhk = df_dq[df_dq['实付金额'] >= df_dq['应付金额']]
dd_dq_yhk = dq_yhk['订单号'].count()
dd_dq_yhk

np.int64(157)

## 到期未还款

In [37]:
dq_whk = df_dq[df_dq['实付金额'] < df_dq['应付金额']]
dd_dq_whk = dq_whk['订单号'].count()
dd_dq_whk

np.int64(44)

# 订单汇总

In [38]:

dd_all = pd.DataFrame({
    '截止日期': [Today1],
    '本月到期订单': [dd_dq_month],
    '今日到期订单': [dd_dq],
    '逾期1+订单': [dd_yq_1],
    '逾期4+订单': [dd_yq_4],
    '逾期7+订单': [dd_yq_7],
    '逾期15+订单': [dd_yq_15],
    '今日到期已还款订单': [dd_dq_yhk],
    '今日到期未还款订单': [dd_dq_whk]
})
dd_all['本月1+逾期率'] = dd_yq_1/dd_dq_month * 100
dd_all['本月1+逾期率'] = dd_all['本月1+逾期率'].apply(lambda x: f'{x: .2f}%')
dd_all['本月4+逾期率'] = dd_yq_4/dd_dq_month * 100
dd_all['本月4+逾期率'] = dd_all['本月4+逾期率'].apply(lambda x: f'{x: .2f}%')
dd_all['本月7+逾期率'] = dd_yq_7/dd_dq_month * 100
dd_all['本月7+逾期率'] = dd_all['本月7+逾期率'].apply(lambda x: f'{x: .2f}%')
dd_all['本月15+逾期率'] = dd_yq_15/dd_dq_month * 100
dd_all['本月15+逾期率'] = dd_all['本月15+逾期率'].apply(lambda x: f'{x: .2f}%')
dd_all['今日到期已还款率'] = dd_dq_yhk/dd_dq * 100
dd_all['今日到期已还款率'] = dd_all['今日到期已还款率'].apply(lambda x: f'{x:.2f}%')
dd_all['今日到期未还款率'] = dd_dq_whk/dd_dq * 100
dd_all['今日到期未还款率'] = dd_all['今日到期未还款率'].apply(lambda x: f'{x:.2f}%')
dd_all = dd_all[['截止日期','本月到期订单','逾期1+订单','本月1+逾期率','逾期4+订单','本月4+逾期率','逾期7+订单','本月7+逾期率','逾期15+订单','本月15+逾期率','今日到期订单','今日到期已还款订单','今日到期已还款率','今日到期未还款订单','今日到期未还款率']]
dd_all



,截止日期,本月到期订单,逾期1+订单,本月1+逾期率,逾期4+订单,本月4+逾期率,逾期7+订单,本月7+逾期率,逾期15+订单,本月15+逾期率,今日到期订单,今日到期已还款订单,今日到期已还款率,今日到期未还款订单,今日到期未还款率
0,2024-10-14,2911,342,11.75%,178,6.11%,108,3.71%,0,0.00%,201,157,78.11%,44,21.89%


# 设置定时任务

In [39]:
# from apscheduler.schedulers.blocking import BlockingScheduler
# from apscheduler.schedulers.background import BackgroundScheduler

# # hour = 9
# # minute = 10
# # week_name = pd.Timestamp.now().dayofweek
# H = datetime.now().strftime('%H')
# M = datetime.now().strftime('%M')
# Today1 = str(datetime.now().strftime('%Y%m%d%H'))
# def my_job1():
#     # print(f'执行定时任务：现在是每日的{hour}点{minute}分')
#     print(f'执行定时任务：现在是每日的{H}点{M}分')
#     path1 = 'F:/需求/到期订单/'
#     path2 = 'F:/需求/截止本月到期订单/'
#     with pd.ExcelWriter(path1 + f'到期订单_信审_{now_date}-{next_date}.xlsx', engine='openpyxl') as writer:
#         df_xs.to_excel(writer, sheet_name=now_date, index=False)
#     with pd.ExcelWriter(path1 + f'到期订单_催收_{now_date}-{next_date}.xlsx', engine='openpyxl') as writer:
#         df_cs.to_excel(writer, sheet_name=now_date, index=False)
#     with pd.ExcelWriter(path2 + f'本月到期订单截止_{Today1}.xlsx', engine='openpyxl') as writer:
#         dd_all.to_excel(writer, sheet_name='订单汇总', index=False)
#     with pd.ExcelWriter(path2 + f'本月到期订单截止_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
#         df4.to_excel(writer, sheet_name='本月到期订单', index=False)
#         df_dq.to_excel(writer, sheet_name='今日到期订单', index=False)
#         df_yq_1.to_excel(writer, sheet_name='逾期1+订单', index=False)
#         df_yq_4.to_excel(writer, sheet_name='逾期4+订单', index=False)
#         df_yq_7.to_excel(writer, sheet_name='逾期7+订单', index=False)
#         df_yq_15.to_excel(writer, sheet_name='逾期15+订单', index=False)
#         dq_yhk.to_excel(writer, sheet_name='今日到期已还款订单', index=False)
#         dq_whk.to_excel(writer, sheet_name='今日到期未还款订单', index=False)
# scheduler1 = BackgroundScheduler()
# # 添加一个cron任务，每天的16点55分执行
# # scheduler1.add_job(my_job1, 'cron', hour=hour, minute=minute)
# scheduler1.add_job(my_job1, 'cron', hour=H, minute=int(M)+1)

# # 启动scheduler1
# scheduler1.start()
    

# 到期订单（方全龙需求）

In [40]:

# with pd.ExcelWriter(f'F:/需求/方全龙需求/订单_{dd_date}.xlsx', engine='openpyxl') as writer:
#     df5.to_excel(writer, sheet_name=f'订单_{dd_date}', index=False)

# 八派、九州信息进件、逾期数据

## 八派信息

In [41]:
df4.columns

Index(['channel_name', 'activity_name', 'total_describes', 'rejected',
       'result', 'order_id', '订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期',
       '实际还款日期', '应付金额', '实付金额', '逾期天数', 'sort', 'order_method', 'status2',
       '取消原因', '电审拒绝原因', '无法联系原因', '进件', '来源渠道', '拒绝理由', '归属渠道', '是否前置拦截',
       '是否机审强拒', '是否出库前风控强拒', '审核状态', '是否进件', '下单月份', '预计还款月份'],
      dtype='object')

In [42]:
df4[df4['是否进件']==1]['订单号'].count()

np.int64(2911)

In [43]:
df4_bp = df4[df4['归属渠道']=='八派信息']
df4_bp_count = df4_bp['订单号'].count()
df4_bp_jj = df4_bp[df4_bp['是否进件']==1]
df4_bp_jj = df4_bp_jj[['订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '实际还款日期', '应付金额', '实付金额']]
df4_bp_jj_count = df4_bp_jj['订单号'].count()

In [44]:
df4_bp_yq = df4_bp[df4_bp['逾期天数']>=1]
df4_bp_yq = df4_bp_yq[['订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '实际还款日期', '应付金额', '实付金额']]
df4_bp_yq_count = df4_bp_yq['订单号'].count()
df4_bp_yq

,订单号,姓名,手机号,产品名称,下单日期,预计还款日期,实际还款日期,应付金额,实付金额
1001,A20240801165244234,何丹丹,15215687070,iPhone15ProMax 全新国行 顺丰包邮,2024-08-01 16:52:45,2024-10-04,NaT,1082.84,0.00
1114,A20240803155541213,纪娜,13240431154,iPhone15 全新国行 6.1英寸屏幕,2024-08-03 15:55:42,2024-10-06,NaT,757.97,0.00
1202,A2024080510132943,周锋,18159190162,iPhone15ProMax 全新国行 顺丰包邮,2024-08-05 10:13:30,2024-10-08,2024-10-09 15:25:39,1082.84,1082.84
1351,A20240807113237121,陈泉,18990984216,iPhone15Pro 全新国行 顺丰包邮,2024-08-07 11:32:38,2024-10-10,NaT,974.55,0.00
1479,A2024080910232583,李军红,18169156121,iPhone15ProMax 全新国行 顺丰包邮,2024-08-09 10:23:25,2024-10-12,NaT,1082.84,0.00
1670,A2024082914041119,辛娜,18669812832,iPhone15ProMax 全新国行 顺丰包邮,2024-08-29 14:04:11,2024-10-01,NaT,1082.84,0.00
1680,A20240829152337114,许晓彤,15751451279,iPhone15ProMax 降价了 全新国行 顺丰包邮 低租金高通过,2024-08-29 15:23:38,2024-10-01,2024-10-02 09:56:21,627.85,627.85
1793,A2024083107552465,何金英,15992138503,iPhone15ProMax 降价了 全新国行 顺丰包邮 低租金高通过,2024-08-31 07:55:24,2024-10-03,NaT,627.85,0.00
1961,A20240902122005103,王睿,15790806674,iPhone15ProMax 全新国行 顺丰包邮,2024-09-02 12:20:06,2024-10-05,NaT,1082.84,0.00
2264,A2024090611182986,张瑞杰,13916000798,iPhone15 全新国行 6.1英寸屏幕,2024-09-06 11:18:29,2024-10-09,NaT,757.97,0.00


In [45]:
df4_bp_rate = pd.DataFrame({
    '渠道': ['八派信息'],
    '总订单数': [df4_bp_count],
    '进件订单数': [df4_bp_jj_count],
    '逾期订单数': [df4_bp_yq_count]
})
df4_bp_rate

,渠道,总订单数,进件订单数,逾期订单数
0,八派信息,55,55,10


## 九州信息

In [46]:
df4_jz = df4[df4['归属渠道']=='九州信息']
df4_jz_jj = df4_jz[df4_jz['是否进件']==1]
df4_jz_jj = df4_jz_jj[['订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '实际还款日期', '应付金额', '实付金额']]

In [47]:
df4_jz_yq = df4_jz[df4_jz['逾期天数']>=1]
df4_jz_yq = df4_jz_yq[['订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '实际还款日期', '应付金额', '实付金额']]
df4_jz_yq

,订单号,姓名,手机号,产品名称,下单日期,预计还款日期,实际还款日期,应付金额,实付金额
2010,A2024090220291563,苏义付,18095377718,iPhone15ProMax 全新国行 顺丰包邮,2024-09-02 20:29:15,2024-10-05,2024-10-06 21:26:34,1082.84,1082.84


In [48]:

# with pd.ExcelWriter(f'F:/需求/八派、九州/进件逾期数据_{Today1}.xlsx', engine='openpyxl') as writer:
#     df4_bp_jj.to_excel(writer, sheet_name='八派信息进件数据', index=False)
# with pd.ExcelWriter(f'F:/需求/八派、九州/进件逾期数据_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
#     df4_bp_yq.to_excel(writer, sheet_name='八派信息逾期数据', index=False)
#     df4_jz_jj.to_excel(writer, sheet_name='九州信息进件数据', index=False)
#     df4_jz_yq.to_excel(writer, sheet_name='九州信息逾期数据', index=False)